In [ ]:
!unzip -o data/data36219/text8.zip

Archive:  data/data36219/text8.zip
  inflating: text8                   


# 章节 1/2：缔造20年经典的LSTM结构

参考文献：

[LONG SHORT-TERM MEMORY](https://www.deepdyve.com/lp/mit-press/long-short-term-memory-ZrphqnW1El)

参考链接：

[【译】理解LSTM（通俗易懂版）](https://www.jianshu.com/p/95d5c461924c)

参考Paddle项目：

[基于LSTM的语言模型实现](https://aistudio.baidu.com/aistudio/projectdetail/592038)


LSTM全程Long Short-Term Memory， 其最早由J. Hochreiter等人在1997年提出。这个经典的模型直到现在仍然被广泛使用和关注，很多研究者基于这个结构又提出了不同的改进结构，下面要说到的GRU就是其一。本项目将简单梳理该模型的原理，并基于Paddle框架进行实现。

**注：本项目的图片来源为[【译】理解LSTM（通俗易懂版）](https://www.jianshu.com/p/95d5c461924c)**

**本人的研究方向是基于复杂网络的文本处理方法，也在致力于探究与深度学习相结合的方法。我会不定期更新自己的工作，如果有研究方向相同的，或者感兴趣的朋友，欢迎三连支持一下。[来AI Studio互粉吧~等你哦~](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/300157)**

### **更多项目链接点击[没入门的研究生的项目合集](https://aistudio.baidu.com/aistudio/projectdetail/542430)**

## 1.1 LSTM结构
LSTM是一种特殊的递归神经网络（RNN）。RNN由于自身结构的特点而非常适用于处理序列数据，其在自然语言处理，语音识别等序列数据处理上有非常成功的应用。LSTM的提出解决了RNN结构的“长依赖”问题。“长依赖”就是词语间跨越很长距离的关联关系。比如，当我们在前文知道某个人现在在北京时，后面推测该人在哪个国家时，会立马得到“他在中国”这个结论。在LSTM提出之前，RNN结构由于参数冲突（详见1.2）的原因，“长依赖”问题一直没有得到解决。不过，好在LSTM出现了。

在一个标准的RNN结构中，网络的输出会与下一步的输入组合，形成新的输入一同送到下一步循环中进行训练。这一典型的自循环结构可以如下图表示：

![](https://ai-studio-static-online.cdn.bcebos.com/0fcdb6c5e1ef4e688e403a20a41863d487503258de5c471ea14e16b6be25e425)

其中左边是真实的结构，右边是为了理解对各个时间步上的结构进行的切片展示。可以看到，每一个时间步上的输出都指向了下一个时间步的输入。这种对数据的处理反映了时序数据一种特点：后边的状态的数据会受到先前状态的影响。

数据在图中A部分可以是经过一个非常简单的处理后被输出，如经过一个tanh层。但是这种简单的处理会面临一些复杂的问题，比如参数的学习会陷入一种无序的状态而不能收敛，甚至造成梯度爆炸而出现喜闻乐见的满屏NAN。LSTM的结构复杂一些，如下图表示：

![](https://ai-studio-static-online.cdn.bcebos.com/b4b509dd74284320a954fef46015eaf47994ec6d21024bca8f4fe062e8dd763f)

各个符号的含义如下：

![](https://ai-studio-static-online.cdn.bcebos.com/c10af4f60e574478a1148b9c326ab3c978af261b590d420189088ec43958b3bd)

从上图我们首先可以看出，LSTM构造了一个自始至终都存在的状态$C_t$，称为细胞状态，如下图所示高亮部分显示。它的存在保证了学习到的信息能够被保留下来，而不会因为后续学习的干扰而被彻底改变。

![](https://ai-studio-static-online.cdn.bcebos.com/cd278551320b4745bdcf5292ed4b438d276c1ca08dc74695a42b15e3e01e70bd)

同时，通过一种被称为门(Gate)的结构，细胞状态$C_t$可以进行更新。门结构其实就是一种点乘操作，将一个和$C_t$等长的状态变量和$C_t$相乘，如果状态变量的某值为0，则对应$C_t$部分与之相乘的结果也为0，这就像是一扇门拦住了$C_t$对应部分不让其通过一样，如上图浅色部分所示。在每个时间步上，$C_t$的状态通过以下方式进行更新：

$C_t=f_t * C_{t-1}+i_t * \hat{C_t}$

我们可以将$f_t$和$i_t$视作两种门：$f_t$直接作用在$C_{t-1}$上，它控制了旧的细胞状态的哪一部分被保留，哪一部分被遗忘；$i_t$则作用在新的细胞状态$\hat{C_t}$上，注意这个细胞状态上面的帽子，区别于旧的细胞状态，这个状态是根据输入而得到的，$i_t$控制了新的细胞状态哪一部分被保留，哪一部分被遗忘。可以看出，$f_t$的作用是忘记旧的知识，$i_t$的作用是学习新的知识，而新的知识来自于对新的输入的学习。下边给出了$f_t,i_t,\hat{C_t}$是如何得到的：

$f_t = \sigma\left(W_f\cdot\left[h_{t-1},x_t\right]+b_f\right)$

$i_t = \sigma\left(W_i\cdot\left[h_{t-1},x_t\right]+b_i\right)$

$\hat{C_t}=tanh\left(W_C\cdot\left[h_{t-1},x_t\right]+b_C\right)$

观察上述三个式子，我们可以发现，它们都是根据当前的输入而进行的操作，其中$f_t$根据当前的输入决定忘记什么，$i_t$根据当前的输入决定记住什么，而$\hat{C_t}$便是对当前输入信息的提炼。明白了上面所述，便明白了为什么$f_t,i_t$中要用激活函数$\sigma$，而$\hat{C_t}$中要用激活函数$tanh$了。那么有些人会问，既然三种值的操作是一样的，为什么不将它们合并成一个值，类似于下式这样，岂不是参数更少而训练效率更高？其实这就是LSTM的创新之处。下式为什么不行呢？请参考1.2中的分析。

$C_t=tanh\left(f\left(C_t,W_C\left[h_{t-1},x_t\right]+b_C\right)\right)$

当细胞状态更新完以后，LSTM也不是直接将该细胞状态输入，而是像上边所述一样，又进行了一次选择性遗忘，套路也一样，式子和上边相同：

$o_t = \sigma\left(W_o\left[h_{t-1},x_t\right]+b_o\right)$

$h_t = o_t * tanh\left(C_t\right)$

至此，LSTM在一个时间步上的动作便完成了，下一步将接收来自上一步的$h_t$和下一步的输入$x_{t+1}$继续重复上述操作。

## 1.2 为何LSTM是这种结构？
对于RNN结构的改进经历了很长的一段历史。在RNN结构被提出来以后，研究人员遭遇了很多困境，其中比较显著的问题是RNN结构的梯度传播问题。由于RNN独特的递归循环的结构，每次迭代中参数的更新都要在如何进行中做抉择：参数既要保留历史的信息，又要依据当前的新输入进行更替。在这个过程中会出现两种冲突：

1. 输入参数冲突。假设我们当前要更新参数$w_{ji}$，其中$i$是输入的某维，$j$是待更新参数的某维。$j$会根据输入$i$来进行更新，以降低整体的误差。在这里，$i$一般不为0。在更新的过程中，$w_{ji}$会收到两种信号：（1）更新$w_{ji}$来储存新的输入信息；（2）保持$w_{ji}$不变以保护历史信息。这种冲突导致了RNN对输入非常敏感，一种轻微的数据扰动，可能就会造成梯度爆炸

2. 输出参数冲突。现在我们假设当前更新参数$w_{kj}$，其中$k$是更新参数的某维，$j$是输出的某维。该参数需要提取输出$j$的信息，同时要避免$j$干扰参数$k$。同样地，$j$一般不为0。在参数更新中，更新信号表现为两种形式：（1）该信号希望参数$w_{kj}$得到$j$的信息；（2）该信号希望避免$j$的信息干扰$k$。

简言之，递归循环的过程中，由于同时涉及到参数的保护和更新，而这种机制是不固定的，很难判断哪个参数需要更新，哪个参数需要保护，在训练过程中便很容易出现波动而使已经训练好的参数重新被扰乱。

lstm的这种结构，就是为了解决上述问题。首先，为了解决梯度更新中梯度爆炸或者梯度消失的问题，引入了一个常数误差（Constant Error Carrousel, CEC），就是结构图示中，上边那个贯穿整个元胞的直线（$C_{t-1}->C_t$）；其次，引入一个输入门（Input Gate Unit）来决定哪些新的信息需要继承；最后，引入一个输出门（Output Gate Unit）来决定何种状态需要被输出。

这种思路导致的结果是，信息的保存和更新这两个过程被分开了。信息的保存通过一个门结构来进行，信息的更新通过另一个门结构来进行，两者分别独享参数，从而避免了上述的两种冲突（当然，是否真正避免了，也不好说）。

## 1.3 用Paddle框架中的lstm函数搞事情
目前以下工具集成了训练（train）,测试（evaluate）功能。以后可能会加上一些更加复杂的玩意，这里先留个坑。

In [ ]:
from paddle import fluid
import numpy as np
import time


class LSTM:
    """
    emb_dim: 词向量维度
    vocab_size: 词典大小，不能小于训练数据中所有词的总数
    num_layers: 隐含层的数量
    hidden_size: 隐含层的大小
    num_steps: LSTM 一次接收数据的最大长度，样本的timestamp
    use_gpu: 是否使用gpu进行训练
    dropout_prob: 如果大于0，就启用dropout，值在0-1区间
    init_scale: 训练参数的初始化范围
    lr：学习速率
    vocab: 默认为None，占位，暂时没用
    """

    def __init__(self,
                 vocab_size,
                 num_layers,
                 hidden_size,
                 num_steps,
                 use_gpu=True,
                 dropout_prob=None,
                 init_scale=0.1,
                 lr=0.001,
                 vocab=None):
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_steps = num_steps
        self.dropout_prob = dropout_prob
        self.use_gpu = use_gpu
        self.init_scale = init_scale
        self.vocab = vocab
        self.lr = lr

    def forward(self, x, batch_size):
        self.init_hidden = fluid.layers.data(name='init_hidden',
                                             shape=[self.num_layers, batch_size, self.hidden_size],
                                             append_batch_size=False)
        self.init_cell = fluid.layers.data(name='init_cell',
                                           shape=[self.num_layers, batch_size, self.hidden_size],
                                           append_batch_size=False)
        x_emb = fluid.embedding(input=x, size=[self.vocab_size, self.hidden_size],
                                dtype='float32', is_sparse=False,
                                param_attr=fluid.ParamAttr(
                                    name='embedding_para',
                                    initializer=fluid.initializer.UniformInitializer(
                                        low=-self.init_scale, high=self.init_scale
                                    )
                                ))
        x_emb = fluid.layers.reshape(x_emb, shape=[-1, self.num_steps, self.hidden_size])
        if self.dropout_prob is not None and self.dropout_prob > 0.0:
            x_emb = fluid.layers.dropout(x_emb, dropout_prob=self.dropout_prob,
                                         dropout_implementation="upscale_in_train")

        rnn_out, last_hidden, last_cell = fluid.contrib.layers.basic_lstm(x_emb, self.init_hidden, self.init_cell,
                                                                          self.hidden_size, self.num_layers,
                                                                          dropout_prob=self.dropout_prob)
        rnn_out = fluid.layers.reshape(rnn_out, shape=[-1, self.num_steps, self.hidden_size])
        softmax_weight = fluid.layers.create_parameter(
            [self.hidden_size, self.vocab_size],
            dtype="float32",
            name="softmax_weight",
            default_initializer=fluid.initializer.UniformInitializer(
                low=-self.init_scale, high=self.init_scale))
        softmax_bias = fluid.layers.create_parameter(
            [self.vocab_size],
            dtype="float32",
            name='softmax_bias',
            default_initializer=fluid.initializer.UniformInitializer(
                low=-self.init_scale, high=self.init_scale))

        proj = fluid.layers.matmul(rnn_out, softmax_weight)
        #proj = fluid.layers.elementwise_add(proj, softmax_bias)
        proj =
        proj = fluid.layers.reshape(proj, shape=[-1, self.vocab_size], inplace=True)
        # 更新 init_hidden, init_cell
        fluid.layers.assign(input=last_cell, output=self.init_cell)
        fluid.layers.assign(input=last_hidden, output=self.init_hidden)
        return proj, last_hidden, last_cell

    def train(self, x, epochs=3, batch_size=32, log_interval=100):
        """
        :param log_interval: 输出信息的间隔
        :param x: 输入，一维list，文本需要经过编码
        :param epochs: 训练回合数
        :param batch_size: 训练batch大小
        :return:
        """
        self.batch_size = batch_size
        # 定义训练的program
        main_program = fluid.default_main_program()
        startup_program = fluid.default_startup_program()
        train_loss, train_proj, self.last_hidden, self.last_cell, py_reader = self.build_train_model(main_program, startup_program)

        # 定义测试的program, 写成全局的，以便留给测试函数
        self.test_program = fluid.Program()
        self.test_startup_program = fluid.Program()
        self.test_loss, self.test_proj, _, _ = self.build_test_model(self.test_program, self.test_startup_program)
        self.test_program = self.test_program.clone(for_test=True)

        place = fluid.CUDAPlace(0) if self.use_gpu else fluid.CPUPlace()
        self.exe = fluid.Executor(place)
        self.exe.run(startup_program)

        def data_gen():
            batches = self.get_data_iter(x)
            for batch in batches:
                x_, y_ = batch
                yield x_, y_

        py_reader.decorate_tensor_provider(data_gen)

        for epoch in range(epochs):
            batch_times = []
            epoch_start_time = time.time()
            total_loss = 0
            iters = 0
            py_reader.start()
            batch_id = 0
            batch_start_time = time.time()
            # 初始化init_hidden, init_cell
            init_hidden = np.zeros((self.num_layers, self.batch_size, self.hidden_size), dtype='float32')
            init_cell = np.zeros((self.num_layers, self.batch_size, self.hidden_size), dtype='float32')

            data_len = len(x)
            batch_len = data_len // self.batch_size
            batch_num = (batch_len - 1) // self.num_steps

            # 送入数据，抓取结果
            try:
                while True:
                    # 送入数据
                    data_feeds = {}
                    data_feeds['init_hidden'] = init_hidden
                    data_feeds['init_cell'] = init_cell
                    fetch_outs = self.exe.run(main_program, feed=data_feeds,
                                         fetch_list=[train_loss.name, self.last_hidden.name, self.last_cell.name])
                    t_loss = np.array(fetch_outs[0])
                    init_hidden = np.array(fetch_outs[1])
                    init_cell = np.array(fetch_outs[2])

                    total_loss += t_loss
                    batch_time = time.time() - batch_start_time
                    batch_times.append(batch_time)
                    batch_start_time = time.time()

                    batch_id += 1
                    iters += self.num_steps
                    if batch_id % log_interval == 0:
                        ppl = np.exp(total_loss / iters)
                        print("-- Epoch: %d - Batch: %d / %d - Cost Time: %.2f s -ETA: %.2f s- ppl: %.5f"
                              % (epoch + 1, batch_id, batch_num, sum(batch_times),
                                 sum(batch_times) / batch_id * (batch_num - batch_id), ppl[0]))
            except fluid.core.EOFException:
                py_reader.reset()

            epoch_time = time.time() - epoch_start_time
            ppl = np.exp(total_loss / iters)
            print("Epoch %d Done. Cost Time: %.2f s. ppl: %.5f." % (epoch + 1, epoch_time, ppl))

    def evaluate(self, x):
        """
        测试模型的效果
        :param x:
        :return:
        """
        eval_data_gen = self.get_data_iter(x)
        total_loss = 0.0
        iters = 0
        # 初始化init_hidden, init_cell
        init_hidden = np.zeros((self.num_layers, self.batch_size, self.hidden_size), dtype='float32')
        init_cell = np.zeros((self.num_layers, self.batch_size, self.hidden_size), dtype='float32')

        for batch_id, batch in enumerate(eval_data_gen):
            x, y = batch
            data_feeds = {}
            data_feeds['init_hidden'] = init_hidden
            data_feeds['init_cell'] = init_cell
            data_feeds['x'] = x
            data_feeds['y'] = y
            fetch_outs = self.exe.run(self.test_program, feed=data_feeds,
                                      fetch_list=[self.test_loss.name, self.last_hidden.name, self.last_cell.name])
            cost_test = np.array(fetch_outs[0])
            init_hidden = np.array(fetch_outs[1])
            init_cell = np.array(fetch_outs[2])

            total_loss += cost_test
            iters += self.num_steps
            ppl = np.exp(total_loss / iters)
            print("-- Batch: %d - ppl: %.5f" % (batch_id, ppl[0]))
        print("ppl: %.5f" % (ppl[0]))
        return ppl

    def get_data_iter(self, raw_data):
        """
        处理原始文本，生成训练数据
        对于RNN来说，一般为读取前n个词，然后预测下一个词，这里简化为每读一个词，预测下一个词。
        由于LSTM考虑了长依赖，所以也可以做到读取n个词，预测下一个词
        :param raw_data: 一个一维数组，list，
        :return:
        """
        data_len = len(raw_data)
        raw_data = np.asarray(raw_data, dtype='int64')
        batch_len = data_len // self.batch_size
        # 将一维数组变为二维数组，第一维是batch的数量，第二维是每个batch的数据，这里对后边不足batch_len的数据进行了裁剪，弃掉不用
        data = raw_data[0:self.batch_size * batch_len].reshape((self.batch_size, batch_len))

        # 为了保证每个batch最后一个词能够被预测，x的词最多被分到batch_len-1
        batch_num = (batch_len - 1) // self.num_steps
        for i in range(batch_num):
            x = np.copy(data[:, i * self.num_steps:(i + 1) * self.num_steps])
            y = np.copy(data[:, i * self.num_steps + 1:(i + 1) * self.num_steps + 1])
            x = x.reshape((-1, self.num_steps, 1))
            y = y.reshape((-1, 1))
            yield x, y

    def build_train_model(self, main_program, startup_program):
        """
        读取数据，构建网络
        :param main_program:
        :param startup_program:
        :return:
        """
        with fluid.program_guard(main_program, startup_program):
            feed_shapes = [[self.batch_size, self.num_steps, 1],
                           [self.batch_size * self.num_steps, 1]]
            py_reader = fluid.layers.py_reader(capacity=64, shapes=feed_shapes, dtypes=['int64', 'int64'])
            x, y = fluid.layers.read_file(py_reader)
            # 使用unique_name.guard创建变量空间，以便在test时共享参数
            with fluid.unique_name.guard():
                proj, last_hidden, last_cell = self.forward(x, self.batch_size)

                loss = self.get_loss(proj, y)
                optimizer = fluid.optimizer.Adam(learning_rate=self.lr,
                                                 grad_clip=fluid.clip.GradientClipByGlobalNorm(clip_norm=1000))
                optimizer.minimize(loss)

                # 不知道有什么用，先写上
                #loss.persistable = True
                #proj.persistable = True
                #last_cell.persistable = True
                #last_hidden.persistable = True

                return loss, proj, last_hidden, last_cell, py_reader

    def build_test_model(self, main_program, startup_program):
        """
        验证模型效果
        :param main_program:
        :param startup_program:
        :return:
        """
        with fluid.program_guard(main_program, startup_program):
            x = fluid.layers.data(name='x', shape=[self.batch_size, self.num_steps, 1], dtype='int64', append_batch_size=False)
            y = fluid.layers.data(name='y', shape=[self.batch_size * self.num_steps, 1], dtype='int64', append_batch_size=False)
            # 使用unique_name.guard创建变量空间，和train共享参数
            with fluid.unique_name.guard():
                proj, last_hidden, last_cell = self.forward(x, self.batch_size)
                loss = self.get_loss(proj, y)

                # 不知道有什么用，先写上
                #loss.persistable = True
                #proj.persistable = True
                #last_cell.persistable = True
                #last_hidden.persistable = True

                return loss, proj, last_hidden, last_cell

    def get_loss(self, proj, y):
        loss = fluid.layers.softmax_with_cross_entropy(logits=proj, label=y, soft_label=False)
        loss = fluid.layers.reshape(loss, shape=[-1, self.num_steps])
        loss = fluid.layers.reduce_mean(loss, dim=[0])
        loss = fluid.layers.reduce_sum(loss)
        return loss

### 1.3.1 数据预处理

In [ ]:
import re
from collections import Counter
import itertools

def clean_str(string):
    """
    将文本中的特定字符串做修改和替换处理
    :param string:
    :return:
    """
    string = re.sub(r"[^A-Za-z0-9:(),!?\'\`]", " ", string)
    string = re.sub(r":", " : ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def build_vocab(sentences, EOS='</eos>'):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    # vocabulary_inv=['<PAD/>', 'the', ....]
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    # vocabulary = {'<PAD/>': 0, 'the': 1, ',': 2, 'a': 3, 'and': 4, ..}
    vocabulary = {x: i+1 for i, x in enumerate(vocabulary_inv)}
    vocabulary[EOS] = 0
    return [vocabulary, vocabulary_inv]


def file_to_ids(src_file, src_vocab):
    """
    将文章单词序列转化成词典id序列
    :param src_file:
    :param src_vocab:
    :return:
    """
    src_data = []
    for line in src_file:
        ids = [src_vocab[w] for w in line if w in src_vocab]
        src_data += ids + [0]
    return src_data

In [ ]:
x_text = list(open("text8", "r").readlines())
x_text = [clean_str(sent) for sent in x_text]
vocabulary, vocabulary_inv = build_vocab(x_text)
x_text = file_to_ids(x_text, vocabulary)

### 1.3.2 训练 
训练的结果与数据的吻合程度用困惑度指标ppl来衡量，参考了[基于LSTM的语言模型实现](https://aistudio.baidu.com/aistudio/projectdetail/592038)。ppl的值即e为底，平均交叉熵损失为指数的幂指数值。

In [5]:
lstm_test = LSTM(vocab_size=len(vocabulary), num_layers=1, hidden_size=100, num_steps=20, use_gpu=True, dropout_prob=0.2, init_scale=0.1, lr=0.01)
lstm_test.train(x_text[:1000000], epochs=3, batch_size=32, log_interval=100)

2020-07-02 10:07:11,468-WARNING: paddle.fluid.layers.py_reader() may be deprecated in the near future. Please use paddle.fluid.io.DataLoader.from_generator() instead.


-- Epoch: 1 - Batch: 100 / 1562 - Cost Time: 3.87 s -ETA: 56.53 s- ppl: 11.70567
-- Epoch: 1 - Batch: 200 / 1562 - Cost Time: 7.03 s -ETA: 47.89 s- ppl: 9.77507
-- Epoch: 1 - Batch: 300 / 1562 - Cost Time: 10.19 s -ETA: 42.88 s- ppl: 8.87627
-- Epoch: 1 - Batch: 400 / 1562 - Cost Time: 13.30 s -ETA: 38.64 s- ppl: 8.27841
-- Epoch: 1 - Batch: 500 / 1562 - Cost Time: 17.05 s -ETA: 36.21 s- ppl: 7.90618


KeyboardInterrupt: 

### 1.3.3 测试结果

In [ ]:
ppl = lstm_test.evaluate(x_text[1000000:1005000])

# 章节 2/2：双门合一的GRU

参考文献：

[Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation](https://arxiv.org/pdf/1406.1078v3.pdf)

理解了章节1中的LSTM模型，GRU模型就容易理解的多了。

## 2.1 GRU的结构
在章节1中我们提到了细胞状态的两种更新方式：旧知识的忘记和新知识的学习，也分析了为什么不能将状态更新中的参数共享。虽然如此，爱问知识人仍然有一个疑问，这几个门就不能简化一下合并成一个门吗？答案是可以。GRU就把$f_t$和$i_t$两个门合并成为了一个门。同时，细胞状态被直接以耦合的形式作为了输出，比LSTM的结构更加的简单了。

![](https://ai-studio-static-online.cdn.bcebos.com/1d281018ac1c46e0acd12193ce0a5df99aa6c9a6e8434d03bb162d8270ddde21)

如下是合并后的门以及细胞状态的更新：

$r_t=\sigma\left(W_r\cdot\left[h_{t-1},x_t\right]\right)$

$\hat{h_t}=tanh\left(W\cdot\left[r_t * h_{t-1},x_t\right]\right)$

如下是细胞状态的更新：

$z_t=\sigma\left(W_z\cdot\left[h_{t-1},x_t\right]\right)$

$h_t=\left(1-z_t\right) * h_{t-1}+z_t * \hat{h_t}$

对比LSTM的变化，我们可以把$h_t$视为$C_t$，把$\hat{h_t}$视为$\hat{C_t}$。我们会很轻易发现，细胞状态直接作为输入和输出了，同时其更新用$z_t$进行了耦合，旧知识被保留的部分，新知识被遗弃，而旧知识被遗弃的部分，新知识被保留。

## 2.2 用Paddle框架中的gru函数搞事情
该工具的形式同上。由于GRU将细胞状态和隐藏状态进行了合并，所以代码中可以直接去掉cell这个变量，只保留hidden。

运行以下程序之前，先重启清除所有内存文件，再运行以下的代码。两个代码之间有冲突，同时运行会造成错误。

In [ ]:
from paddle import fluid
import numpy as np
import time

class GRU:
    """
    vocab_size: 词典大小，不能小于训练数据中所有词的总数
    num_layers: 隐含层的数量
    hidden_size: 隐含层的大小
    num_steps: LSTM 一次接收数据的最大长度，样本的timestamp
    use_gpu: 是否使用gpu进行训练
    dropout_prob: 如果大于0，就启用dropout，值在0-1区间
    init_scale: 训练参数的初始化范围
    lr：学习速率
    vocab: 默认为None，占位，暂时没用
    """

    def __init__(self,
                 vocab_size,
                 num_layers,
                 hidden_size,
                 num_steps,
                 use_gpu=True,
                 dropout_prob=None,
                 init_scale=0.1,
                 lr=0.001,
                 vocab=None):
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_steps = num_steps
        self.dropout_prob = dropout_prob
        self.use_gpu = use_gpu
        self.init_scale = init_scale
        self.vocab = vocab
        self.lr = lr

    def forward(self, x, batch_size):
        self.init_hidden = fluid.layers.data(name='init_hidden',
                                             shape=[self.num_layers, batch_size, self.hidden_size],
                                             append_batch_size=False)
        x_emb = fluid.embedding(input=x, size=[self.vocab_size, self.hidden_size],
                                dtype='float32', is_sparse=False,
                                param_attr=fluid.ParamAttr(
                                    name='embedding_para',
                                    initializer=fluid.initializer.UniformInitializer(
                                        low=-self.init_scale, high=self.init_scale
                                    )
                                ))
        x_emb = fluid.layers.reshape(x_emb, shape=[-1, self.num_steps, self.hidden_size])
        if self.dropout_prob is not None and self.dropout_prob > 0.0:
            x_emb = fluid.layers.dropout(x_emb, dropout_prob=self.dropout_prob,
                                         dropout_implementation="upscale_in_train")

        rnn_out, last_hidden, = fluid.contrib.layers.basic_gru(x_emb, self.init_hidden, self.hidden_size,
                                                                self.num_layers, dropout_prob=self.dropout_prob)
        rnn_out = fluid.layers.reshape(rnn_out, shape=[-1, self.num_steps, self.hidden_size])
        softmax_weight = fluid.layers.create_parameter(
            [self.hidden_size, self.vocab_size],
            dtype="float32",
            name="softmax_weight",
            default_initializer=fluid.initializer.UniformInitializer(
                low=-self.init_scale, high=self.init_scale))
        softmax_bias = fluid.layers.create_parameter(
            [self.vocab_size],
            dtype="float32",
            name='softmax_bias',
            default_initializer=fluid.initializer.UniformInitializer(
                low=-self.init_scale, high=self.init_scale))

        proj = fluid.layers.matmul(rnn_out, softmax_weight)
        proj = fluid.layers.elementwise_add(proj, softmax_bias)
        proj = fluid.layers.reshape(proj, shape=[-1, self.vocab_size], inplace=True)
        # 更新 init_hidden, init_cell
        fluid.layers.assign(input=last_hidden, output=self.init_hidden)
        return proj, last_hidden

    def train(self, x, epochs=3, batch_size=32, log_interval=100):
        """
        :param log_interval: 输出信息的间隔
        :param x: 输入，一维或者二维的文本list，文本需要经过编码
        :param epochs: 训练回合数
        :param batch_size: 训练batch大小
        :return:
        """
        self.batch_size = batch_size
        # 定义训练的program
        main_program = fluid.default_main_program()
        startup_program = fluid.default_startup_program()
        train_loss, train_proj, self.last_hidden, py_reader = self.build_train_model(main_program, startup_program)

        # 定义测试的program, 写成全局的，以便留给测试函数
        self.test_program = fluid.Program()
        self.test_startup_program = fluid.Program()
        self.test_loss, self.test_proj, _, = self.build_test_model(self.test_program, self.test_startup_program)
        self.test_program = self.test_program.clone(for_test=True)

        place = fluid.CUDAPlace(0) if self.use_gpu else fluid.CPUPlace()
        self.exe = fluid.Executor(place)
        self.exe.run(startup_program)

        def data_gen():
            batches = self.get_data_iter(x)
            for batch in batches:
                x_, y_ = batch
                yield x_, y_

        py_reader.decorate_tensor_provider(data_gen)

        for epoch in range(epochs):
            batch_times = []
            epoch_start_time = time.time()
            total_loss = 0
            iters = 0
            py_reader.start()
            batch_id = 0
            batch_start_time = time.time()
            # 初始化init_hidden, init_cell
            init_hidden = np.zeros((self.num_layers, self.batch_size, self.hidden_size), dtype='float32')

            data_len = len(x)
            batch_len = data_len // self.batch_size
            batch_num = (batch_len - 1) // self.num_steps

            # 送入数据，抓取结果
            try:
                while True:
                    # 送入数据
                    data_feeds = {}
                    data_feeds['init_hidden'] = init_hidden
                    fetch_outs = self.exe.run(main_program, feed=data_feeds,
                                         fetch_list=[train_loss.name, self.last_hidden.name])
                    t_loss = np.array(fetch_outs[0])
                    init_hidden = np.array(fetch_outs[1])

                    total_loss += t_loss
                    batch_time = time.time() - batch_start_time
                    batch_times.append(batch_time)
                    batch_start_time = time.time()

                    batch_id += 1
                    iters += self.num_steps
                    if batch_id % log_interval == 0:
                        ppl = np.exp(total_loss / iters)
                        print("-- Epoch: %d - Batch: %d / %d - Cost Time: %.2f s -ETA: %.2f s- ppl: %.5f"
                              % (epoch + 1, batch_id, batch_num, sum(batch_times),
                                 sum(batch_times) / batch_id * (batch_num - batch_id), ppl[0]))
            except fluid.core.EOFException:
                py_reader.reset()

            epoch_time = time.time() - epoch_start_time
            ppl = np.exp(total_loss / iters)
            print("Epoch %d Done. Cost Time: %.2f s. ppl: %.5f." % (epoch + 1, epoch_time, ppl))

    def evaluate(self, x):
        """
        测试模型的效果
        :param x:
        :return:
        """
        eval_data_gen = self.get_data_iter(x)
        total_loss = 0.0
        iters = 0
        # 初始化init_hidden, init_cell
        init_hidden = np.zeros((self.num_layers, self.batch_size, self.hidden_size), dtype='float32')

        for batch_id, batch in enumerate(eval_data_gen):
            x, y = batch
            data_feeds = {}
            data_feeds['init_hidden'] = init_hidden
            data_feeds['x'] = x
            data_feeds['y'] = y
            fetch_outs = self.exe.run(self.test_program, feed=data_feeds,
                                      fetch_list=[self.test_loss.name, self.last_hidden.name])
            cost_test = np.array(fetch_outs[0])
            init_hidden = np.array(fetch_outs[1])

            total_loss += cost_test
            iters += self.num_steps
            ppl = np.exp(total_loss / iters)
            print("-- Batch: %d - ppl: %.5f" % (batch_id, ppl[0]))
        print("ppl: %.5f" % (ppl[0]))
        return ppl

    def get_data_iter(self, raw_data):
        """
        处理原始文本，生成训练数据
        对于RNN来说，一般为读取前n个词，然后预测下一个词，这里简化为每读一个词，预测下一个词。
        由于LSTM考虑了长依赖，所以也可以做到读取n个词，预测下一个词
        :param raw_data: 一个一维数组，list，
        :return:
        """
        data_len = len(raw_data)
        raw_data = np.asarray(raw_data, dtype='int64')
        batch_len = data_len // self.batch_size
        # 将一维数组变为二维数组，第一维是batch的数量，第二维是每个batch的数据，这里对后边不足batch_len的数据进行了裁剪，弃掉不用
        data = raw_data[0:self.batch_size * batch_len].reshape((self.batch_size, batch_len))

        # 为了保证每个batch最后一个词能够被预测，x的词最多被分到batch_len-1
        batch_num = (batch_len - 1) // self.num_steps
        for i in range(batch_num):
            x = np.copy(data[:, i * self.num_steps:(i + 1) * self.num_steps])
            y = np.copy(data[:, i * self.num_steps + 1:(i + 1) * self.num_steps + 1])
            x = x.reshape((-1, self.num_steps, 1))
            y = y.reshape((-1, 1))
            yield x, y

    def build_train_model(self, main_program, startup_program):
        """
        读取数据，构建网络
        :param main_program:
        :param startup_program:
        :return:
        """
        with fluid.program_guard(main_program, startup_program):
            feed_shapes = [[self.batch_size, self.num_steps, 1],
                           [self.batch_size * self.num_steps, 1]]
            py_reader = fluid.layers.py_reader(capacity=64, shapes=feed_shapes, dtypes=['int64', 'int64'])
            x, y = fluid.layers.read_file(py_reader)
            # 使用unique_name.guard创建变量空间，以便在test时共享参数
            with fluid.unique_name.guard():
                proj, last_hidden = self.forward(x, self.batch_size)

                loss = self.get_loss(proj, y)
                optimizer = fluid.optimizer.Adam(learning_rate=self.lr,
                                                 grad_clip=fluid.clip.GradientClipByGlobalNorm(clip_norm=1000))
                optimizer.minimize(loss)

                return loss, proj, last_hidden, py_reader

    def build_test_model(self, main_program, startup_program):
        """
        验证模型效果
        :param main_program:
        :param startup_program:
        :return:
        """
        with fluid.program_guard(main_program, startup_program):
            x = fluid.layers.data(name='x', shape=[self.batch_size, self.num_steps, 1], dtype='int64', append_batch_size=False)
            y = fluid.layers.data(name='y', shape=[self.batch_size * self.num_steps, 1], dtype='int64', append_batch_size=False)
            # 使用unique_name.guard创建变量空间，和train共享参数
            with fluid.unique_name.guard():
                proj, last_hidden = self.forward(x, self.batch_size)
                loss = self.get_loss(proj, y)

                return loss, proj, last_hidden

    def get_loss(self, proj, y):
        loss = fluid.layers.softmax_with_cross_entropy(logits=proj, label=y, soft_label=False)
        loss = fluid.layers.reshape(loss, shape=[-1, self.num_steps])
        loss = fluid.layers.reduce_mean(loss, dim=[0])
        loss = fluid.layers.reduce_sum(loss)
        return loss

### 2.2.1 数据预处理

In [ ]:
import re
from collections import Counter
import itertools

def clean_str(string):
    """
    将文本中的特定字符串做修改和替换处理
    :param string:
    :return:
    """
    string = re.sub(r"[^A-Za-z0-9:(),!?\'\`]", " ", string)
    string = re.sub(r":", " : ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def build_vocab(sentences, EOS='</eos>'):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    # vocabulary_inv=['<PAD/>', 'the', ....]
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    # vocabulary = {'<PAD/>': 0, 'the': 1, ',': 2, 'a': 3, 'and': 4, ..}
    vocabulary = {x: i+1 for i, x in enumerate(vocabulary_inv)}
    vocabulary[EOS] = 0
    return [vocabulary, vocabulary_inv]


def file_to_ids(src_file, src_vocab):
    """
    将文章单词序列转化成词典id序列
    :param src_file:
    :param src_vocab:
    :return:
    """
    src_data = []
    for line in src_file:
        ids = [src_vocab[w] for w in line if w in src_vocab]
        src_data += ids + [0]
    return src_data

In [ ]:
x_text = list(open("text8", "r").readlines())
x_text = [clean_str(sent) for sent in x_text]
vocabulary, vocabulary_inv = build_vocab(x_text)
x_text = file_to_ids(x_text, vocabulary)

### 2.2.2 训练

In [ ]:
gru_test = GRU(vocab_size=len(vocabulary), num_layers=1, hidden_size=100, num_steps=20, use_gpu=True, dropout_prob=0.2, init_scale=0.1, lr=0.01)
gru_test.train(x_text[:1000000], epochs=3, batch_size=32, log_interval=100)

### 2.2.3 测试结果

In [ ]:
ppl = gru_test.evaluate(x_text[1000000:1005000])

# 总结
LSTM作为一个经典的RNN结构，不仅应用于文本信息处理。以上工具也可以直接用于时序数据等纯数据的应用中。GRU是LSTM的改进形式，在保证性能表现的情况下，将LSTM进一步简化，减少了参数数量，也提高了运行效率。Paddle框架提供了经典LSTM和GRU工具，给编程带来了很大的便利，赞一个。